# Module 2: Memory and Learning Systems 🧠
*Building Retail AI Agents That Remember and Improve*

**Duration**: 50 minutes  
**Level**: Intermediate  
**Domain**: Retail Operations (Walmart)

## 🎯 Learning Objectives

By the end of this module, you'll understand:

1. **Memory Architecture**: Four types of memory systems for retail AI
2. **Working Memory**: Real-time inventory and customer interactions
3. **Episodic Memory**: Learning from past sales events and seasons
4. **Semantic Memory**: Product knowledge and store operations
5. **Procedural Memory**: Optimized workflows and best practices
6. **Retail Intelligence**: Using memories for better decision-making
7. **Adaptive Learning**: Improving from Black Friday to everyday ops

## 🏪 Retail Context

Imagine you're building an AI assistant for Walmart that:
- Remembers customer shopping patterns
- Learns from seasonal trends
- Optimizes inventory management
- Improves store operations
- Adapts to local market conditions

## 📚 Understanding Memory Types in Retail

### 1. **Working Memory** (Current Operations)
- Active customer in store needing help
- Current inventory levels and alerts
- Today's promotions and pricing
- Real-time store traffic

### 2. **Episodic Memory** (Past Events)
- Last year's Black Friday performance
- Successful product launches
- Customer complaint resolutions
- Seasonal shopping patterns

### 3. **Semantic Memory** (Retail Knowledge)
- Product specifications and relationships
- Store layout and department info
- Supplier details and lead times
- Walmart policies and procedures

### 4. **Procedural Memory** (How-To Skills)
- Inventory reordering workflows
- Customer service protocols
- Price matching procedures
- Seasonal transition strategies

In [ ]:
# Core imports
import json
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from collections import deque, defaultdict
from enum import Enum
import time
import math
import random

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Walmart Retail AI Memory System Ready!")
print("🏪 Location: Bentonville, AR Supercenter #100")

## 🏗️ Part 1: Retail Memory Architecture

Let's build a memory system specifically designed for retail operations.

In [ ]:
# Define memory types for retail context
class MemoryType(Enum):
    """Types of memory in our retail AI system"""
    WORKING = "working"      # Current store state
    EPISODIC = "episodic"    # Past events and outcomes
    SEMANTIC = "semantic"    # Product and policy knowledge
    PROCEDURAL = "procedural" # Operational workflows

@dataclass
class RetailMemoryItem:
    """Base class for all retail memory items"""
    content: Any                    # The memory content
    timestamp: datetime             # When it occurred
    importance: float               # Business impact (0-1)
    department: str = "General"     # Which department
    store_id: str = "#100"         # Which store
    access_count: int = 0          # How often referenced
    last_accessed: Optional[datetime] = None
    decay_rate: float = 0.1        # How fast it becomes less relevant
    memory_type: MemoryType = MemoryType.EPISODIC
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def access(self):
        """Update access statistics"""
        self.access_count += 1
        self.last_accessed = datetime.now()
        # Accessing reinforces memory importance
        self.importance = min(1.0, self.importance * 1.1)
    
    def get_strength(self) -> float:
        """Calculate memory strength with time decay"""
        if not self.last_accessed:
            self.last_accessed = self.timestamp
            
        time_since_access = (datetime.now() - self.last_accessed).total_seconds()
        # Exponential decay with importance weighting
        decay_factor = math.exp(-self.decay_rate * time_since_access / 3600)  # Hourly decay
        return self.importance * decay_factor * (1 + math.log1p(self.access_count))

@dataclass
class RetailEpisode(RetailMemoryItem):
    """Memory of a specific retail event"""
    event_type: str = ""           # sale, return, complaint, etc.
    products_involved: List[str] = field(default_factory=list)
    customer_segment: str = ""     # regular, premium, new
    outcome: str = ""              # What happened
    revenue_impact: float = 0.0    # Dollar impact
    success: bool = True           # Was it successful?
    
    def __post_init__(self):
        self.memory_type = MemoryType.EPISODIC

@dataclass
class ProductKnowledge(RetailMemoryItem):
    """Semantic memory about products and categories"""
    product_id: str = ""           # SKU or product ID
    category: str = ""             # Product category
    relations: Dict[str, List[str]] = field(default_factory=dict)  # Related products
    attributes: Dict[str, Any] = field(default_factory=dict)  # Price, features, etc.
    supplier_info: Dict[str, Any] = field(default_factory=dict)
    confidence: float = 1.0        # How accurate this info is
    
    def __post_init__(self):
        self.memory_type = MemoryType.SEMANTIC

@dataclass 
class RetailProcedure(RetailMemoryItem):
    """Memory of retail operational procedures"""
    procedure_name: str = ""       # What procedure
    trigger_conditions: List[str] = field(default_factory=list)  # When to use
    steps: List[str] = field(default_factory=list)  # How to execute
    success_rate: float = 0.0      # Historical effectiveness
    execution_count: int = 0       # Times used
    average_time: float = 0.0      # Minutes to complete
    cost_savings: float = 0.0      # Average $ saved/earned
    
    def __post_init__(self):
        self.memory_type = MemoryType.PROCEDURAL
        
    def update_performance(self, success: bool, time_taken: float, savings: float = 0):
        """Update procedure performance metrics"""
        self.execution_count += 1
        # Running average for success rate
        self.success_rate = ((self.success_rate * (self.execution_count - 1)) + 
                           (1.0 if success else 0.0)) / self.execution_count
        # Running average for time
        self.average_time = ((self.average_time * (self.execution_count - 1)) + 
                           time_taken) / self.execution_count
        # Track cost savings
        self.cost_savings = ((self.cost_savings * (self.execution_count - 1)) + 
                           savings) / self.execution_count

@dataclass
class CustomerInteraction(RetailMemoryItem):
    """Special episodic memory for customer interactions"""
    interaction_type: str = ""     # inquiry, complaint, praise
    customer_mood: str = ""        # happy, frustrated, neutral
    resolution: str = ""           # How it was resolved
    satisfaction_score: float = 0.0  # 1-5 rating
    associate_id: str = ""         # Who helped
    
    def __post_init__(self):
        self.memory_type = MemoryType.EPISODIC
        # Customer interactions are always important
        self.importance = max(0.7, self.importance)

print("✅ Retail memory structures defined!")
print(f"Memory types: {[t.value for t in MemoryType]}")
print("\n🏪 Specialized for Walmart operations:")
print("  - Product relationships and inventory")
print("  - Customer interaction tracking")
print("  - Procedural efficiency metrics")
print("  - Revenue impact analysis")

## 🧠 Part 2: Walmart Memory Manager

A sophisticated memory system that handles retail-specific scenarios.

In [ ]:
class WalmartMemoryManager:
    """Manages retail memories with business intelligence"""
    
    def __init__(self, 
                 store_id: str = "#100",
                 working_memory_size: int = 10,
                 consolidation_threshold: float = 0.7):
        self.store_id = store_id
        
        # Working memory - current store state
        self.working_memory = deque(maxlen=working_memory_size)
        
        # Long-term memories
        self.episodes: List[RetailEpisode] = []
        self.product_knowledge: Dict[str, ProductKnowledge] = {}
        self.procedures: Dict[str, RetailProcedure] = {}
        self.customer_interactions: List[CustomerInteraction] = []
        
        # Indices for fast retrieval
        self.product_index = defaultdict(list)  # category -> products
        self.seasonal_index = defaultdict(list)  # season -> episodes
        self.department_index = defaultdict(list)  # dept -> memories
        
        # Business metrics
        self.total_revenue_impact = 0.0
        self.customer_satisfaction_avg = 4.0
        self.consolidation_threshold = consolidation_threshold
        
        # Initialize with some Walmart knowledge
        self._initialize_walmart_knowledge()
    
    def _initialize_walmart_knowledge(self):
        """Pre-load essential Walmart operational knowledge"""
        # Price match procedure
        price_match = RetailProcedure(
            content="Walmart Price Match Guarantee procedure",
            timestamp=datetime.now(),
            importance=0.9,
            department="Customer Service",
            procedure_name="price_match",
            trigger_conditions=["customer shows lower price", "competitor ad"],
            steps=[
                "Verify competitor price on current ad",
                "Check if item is identical (brand, size, model)",
                "Confirm competitor has item in stock",
                "Apply price match at register",
                "Log price match in system"
            ],
            success_rate=0.95,
            execution_count=1000,
            average_time=3.5,
            cost_savings=-5.50  # Average loss per match, but retains customer
        )
        self.procedures["price_match"] = price_match
        
        # Black Friday prep procedure
        black_friday = RetailProcedure(
            content="Black Friday preparation workflow",
            timestamp=datetime.now(),
            importance=1.0,
            department="Operations",
            procedure_name="black_friday_prep",
            trigger_conditions=["2 weeks before Black Friday"],
            steps=[
                "Review last year's sales data and pain points",
                "Double-check inventory levels for doorbusters",
                "Schedule extra staff for all departments",
                "Set up queue management systems",
                "Prepare online pickup area for increased volume",
                "Test all POS systems and backups",
                "Brief staff on crowd control procedures"
            ],
            success_rate=0.92,
            execution_count=5,
            average_time=480,  # 8 hours
            cost_savings=125000  # Smooth operations save/earn this much
        )
        self.procedures["black_friday_prep"] = black_friday
        
        print("📚 Loaded Walmart operational procedures")
    
    def remember_sale(self, products: List[str], amount: float, 
                     customer_type: str = "regular", special_event: str = ""):
        """Store memory of a sale transaction"""
        episode = RetailEpisode(
            content=f"Sale of {len(products)} items for ${amount:.2f}",
            timestamp=datetime.now(),
            importance=min(1.0, amount / 1000),  # Higher sales more important
            department=self._get_department(products[0] if products else "General"),
            event_type="sale",
            products_involved=products,
            customer_segment=customer_type,
            outcome=f"Completed sale${amount:.2f}",
            revenue_impact=amount,
            success=True
        )
        
        if special_event:
            episode.metadata["event"] = special_event
            self.seasonal_index[special_event].append(episode)
            
        self.working_memory.append(episode)
        self.episodes.append(episode)
        self.total_revenue_impact += amount
        
        # Update product relationships
        if len(products) > 1:
            self._update_product_relationships(products)
            
        return episode
    
    def remember_customer_interaction(self, interaction_type: str, 
                                    department: str, resolution: str,
                                    satisfaction: float, mood: str = "neutral"):
        """Store customer service interaction"""
        interaction = CustomerInteraction(
            content=f"{interaction_type} in {department}: {resolution}",
            timestamp=datetime.now(),
            importance=0.8 if interaction_type == "complaint" else 0.6,
            department=department,
            interaction_type=interaction_type,
            customer_mood=mood,
            resolution=resolution,
            satisfaction_score=satisfaction,
            associate_id=f"A{random.randint(1000, 9999)}"
        )
        
        self.working_memory.append(interaction)
        self.customer_interactions.append(interaction)
        
        # Update satisfaction average
        total_interactions = len(self.customer_interactions)
        self.customer_satisfaction_avg = (
            (self.customer_satisfaction_avg * (total_interactions - 1) + satisfaction) 
            / total_interactions
        )
        
        return interaction
    
    def learn_product_fact(self, product_id: str, category: str, 
                          attributes: Dict, relations: Dict[str, List[str]] = None):
        """Store product knowledge"""
        knowledge = ProductKnowledge(
            content=f"Product {product_id} in {category}",
            timestamp=datetime.now(),
            importance=0.7,
            department=self._get_department(category),
            product_id=product_id,
            category=category,
            attributes=attributes,
            relations=relations or {},
            confidence=0.95
        )
        
        # Merge with existing knowledge if exists
        if product_id in self.product_knowledge:
            existing = self.product_knowledge[product_id]
            existing.attributes.update(attributes)
            if relations:
                for rel_type, items in relations.items():
                    if rel_type in existing.relations:
                        existing.relations[rel_type].extend(items)
                    else:
                        existing.relations[rel_type] = items
        else:
            self.product_knowledge[product_id] = knowledge
            self.product_index[category].append(product_id)
            
        return knowledge
    
    def _get_department(self, item: str) -> str:
        """Map items to Walmart departments"""
        dept_map = {
            "grocery": "Grocery",
            "electronics": "Electronics",
            "apparel": "Apparel",
            "home": "Home & Garden",
            "pharmacy": "Pharmacy",
            "auto": "Auto Care",
            "toys": "Toys",
            "sports": "Sports & Outdoors"
        }
        
        item_lower = item.lower()
        for key, dept in dept_map.items():
            if key in item_lower:
                return dept
        return "General Merchandise"
    
    def _update_product_relationships(self, products: List[str]):
        """Learn which products are frequently bought together"""
        for i, product1 in enumerate(products):
            if product1 in self.product_knowledge:
                knowledge = self.product_knowledge[product1]
                if "frequently_bought_with" not in knowledge.relations:
                    knowledge.relations["frequently_bought_with"] = []
                    
                for product2 in products[i+1:]:
                    if product2 not in knowledge.relations["frequently_bought_with"]:
                        knowledge.relations["frequently_bought_with"].append(product2)
    
    def retrieve_relevant_memories(self, query: str, context: Dict[str, Any] = None,
                                 limit: int = 5) -> List[RetailMemoryItem]:
        """Retrieve memories relevant to current retail situation"""
        relevant_memories = []
        query_lower = query.lower()
        
        # Check if it's a seasonal query
        seasonal_keywords = ["black friday", "christmas", "back to school", 
                           "easter", "summer", "winter"]
        for season in seasonal_keywords:
            if season in query_lower and season in self.seasonal_index:
                relevant_memories.extend(self.seasonal_index[season])
        
        # Search episodes
        for episode in self.episodes:
            if any(product.lower() in query_lower 
                  for product in episode.products_involved):
                relevant_memories.append(episode)
        
        # Search product knowledge
        for product_id, knowledge in self.product_knowledge.items():
            if (product_id.lower() in query_lower or 
                knowledge.category.lower() in query_lower):
                relevant_memories.append(knowledge)
        
        # Search procedures
        for proc_name, procedure in self.procedures.items():
            if (proc_name in query_lower or 
                any(trigger.lower() in query_lower 
                    for trigger in procedure.trigger_conditions)):
                relevant_memories.append(procedure)
        
        # Search customer interactions if relevant
        if any(word in query_lower for word in ["customer", "complaint", "service"]):
            relevant_memories.extend(self.customer_interactions[-5:])  # Recent ones
        
        # Remove duplicates and sort by relevance
        unique_memories = list({id(m): m for m in relevant_memories}.values())
        sorted_memories = sorted(unique_memories, 
                               key=lambda m: m.get_strength(), 
                               reverse=True)
        
        # Update access counts
        for memory in sorted_memories[:limit]:
            memory.access()
            
        return sorted_memories[:limit]
    
    def get_department_insights(self, department: str) -> Dict[str, Any]:
        """Get insights for a specific department"""
        dept_episodes = [e for e in self.episodes if e.department == department]
        
        if not dept_episodes:
            return {"message": f"No data for {department}"}
            
        total_revenue = sum(e.revenue_impact for e in dept_episodes)
        avg_transaction = total_revenue / len(dept_episodes) if dept_episodes else 0
        
        # Find best selling products
        product_sales = defaultdict(int)
        for episode in dept_episodes:
            for product in episode.products_involved:
                product_sales[product] += 1
                
        top_products = sorted(product_sales.items(), 
                            key=lambda x: x[1], reverse=True)[:5]
        
        return {
            "department": department,
            "total_revenue": total_revenue,
            "transaction_count": len(dept_episodes),
            "avg_transaction_size": avg_transaction,
            "top_products": top_products,
            "time_range": {
                "earliest": min(e.timestamp for e in dept_episodes),
                "latest": max(e.timestamp for e in dept_episodes)
            }
        }
    
    def visualize_retail_intelligence(self):
        """Create retail-specific memory visualizations"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Walmart Store {self.store_id} Memory Analytics', fontsize=16)
        
        # Department revenue distribution
        dept_revenue = defaultdict(float)
        for episode in self.episodes:
            if episode.revenue_impact > 0:
                dept_revenue[episode.department] += episode.revenue_impact
                
        if dept_revenue:
            depts, revenues = zip(*dept_revenue.items())
            ax1.pie(revenues, labels=depts, autopct='%1.1f%%', startangle=90)
            ax1.set_title("Revenue by Department")
        
        # Customer satisfaction trend
        if self.customer_interactions:
            interactions = self.customer_interactions[-20:]  # Last 20
            satisfaction_scores = [i.satisfaction_score for i in interactions]
            ax2.plot(satisfaction_scores, marker='o', linewidth=2, markersize=8)
            ax2.axhline(y=4.0, color='g', linestyle='--', label='Target (4.0)')
            ax2.set_xlabel("Recent Interactions")
            ax2.set_ylabel("Satisfaction Score")
            ax2.set_title("Customer Satisfaction Trend")
            ax2.set_ylim(1, 5)
            ax2.legend()
        
        # Procedure effectiveness
        if self.procedures:
            procs = list(self.procedures.values())
            proc_names = [p.procedure_name for p in procs]
            success_rates = [p.success_rate for p in procs]
            savings = [p.cost_savings for p in procs]
            
            x = np.arange(len(proc_names))
            width = 0.35
            
            ax3.bar(x - width/2, success_rates, width, label='Success Rate')
            ax3.set_xlabel('Procedures')
            ax3.set_ylabel('Success Rate', color='b')
            ax3.set_title('Procedure Performance')
            ax3.set_xticks(x)
            ax3.set_xticklabels(proc_names, rotation=45, ha='right')
            ax3.set_ylim(0, 1.1)
            
            # Add savings on secondary axis
            ax3_twin = ax3.twinx()
            ax3_twin.bar(x + width/2, savings, width, label='Avg Savings', color='g')
            ax3_twin.set_ylabel('Average Savings ($)', color='g')
        
        # Time-based activity
        if self.episodes:
            # Group by hour of day
            hour_activity = defaultdict(int)
            for episode in self.episodes:
                hour = episode.timestamp.hour
                hour_activity[hour] += 1
                
            hours = list(range(24))
            activity = [hour_activity.get(h, 0) for h in hours]
            
            ax4.bar(hours, activity, color='skyblue')
            ax4.set_xlabel('Hour of Day')
            ax4.set_ylabel('Transaction Count')
            ax4.set_title('Store Activity by Hour')
            ax4.set_xticks(range(0, 24, 2))
        
        plt.tight_layout()
        plt.show()

# Create Walmart memory manager
walmart_memory = WalmartMemoryManager(store_id="#100")
print(f"\n✅ Walmart Memory Manager initialized for Store {walmart_memory.store_id}")
print(f"📊 Customer satisfaction baseline: {walmart_memory.customer_satisfaction_avg:.1f}/5.0")
print(f"📋 Pre-loaded procedures: {list(walmart_memory.procedures.keys())}")

## 🧪 Part 3: Retail Scenarios in Action

Let's simulate real Walmart scenarios to see our memory system work.

In [ ]:
# Scenario 1: Black Friday Shopping Pattern
print("🛒 Simulating Black Friday 2023...\n")

# Early morning doorbusters
black_friday_sales = [
    (["Samsung 65in TV", "Soundbar", "HDMI Cable"], 899.99, "deal_hunter"),
    (["PS5 Console", "Extra Controller", "Game Bundle"], 549.99, "gamer"),
    (["Instant Pot", "Air Fryer", "Kitchen Scale"], 129.99, "regular"),
    (["Apple iPad", "Apple Pencil", "Case"], 449.99, "premium"),
    (["Kids Bike", "Helmet", "Training Wheels"], 89.99, "family"),
]

for products, amount, customer_type in black_friday_sales:
    episode = walmart_memory.remember_sale(
        products=products,
        amount=amount,
        customer_type=customer_type,
        special_event="Black Friday 2023"
    )
    print(f"✅ {episode.content}")

print(f"\n💰 Total Black Friday revenue so far: ${walmart_memory.total_revenue_impact:,.2f}")

In [ ]:
# Scenario 2: Customer Service Interactions
print("\n👥 Customer Service Desk Activity...\n")

interactions = [
    ("complaint", "Electronics", "Exchanged defective headphones", 3.5, "frustrated"),
    ("inquiry", "Grocery", "Located gluten-free products in aisle 7", 5.0, "happy"),
    ("return", "Apparel", "Processed return without receipt per policy", 4.0, "neutral"),
    ("complaint", "Customer Service", "Applied price match to competitor ad", 4.5, "satisfied"),
    ("praise", "Electronics", "Customer thanked associate for TV setup help", 5.0, "delighted"),
]

for int_type, dept, resolution, satisfaction, mood in interactions:
    interaction = walmart_memory.remember_customer_interaction(
        interaction_type=int_type,
        department=dept,
        resolution=resolution,
        satisfaction=satisfaction,
        mood=mood
    )
    emoji = "😊" if satisfaction >= 4 else "😐" if satisfaction >= 3 else "😞"
    print(f"{emoji} [{int_type}] {dept}: {resolution} (Score: {satisfaction}/5)")

print(f"\n📈 Updated satisfaction average: {walmart_memory.customer_satisfaction_avg:.2f}/5.0")

In [ ]:
# Scenario 3: Learning Product Relationships
print("\n📦 Loading Product Knowledge...\n")

# Popular products with relationships
products = [
    ("TV-SAMSUNG-65-Q80A", "Electronics", 
     {"price": 899.99, "brand": "Samsung", "size": "65 inch", "display": "QLED"},
     {"accessories": ["Wall Mount", "HDMI Cable"], "competes_with": ["LG OLED65"]}),
    
    ("FOOD-CEREAL-CHEERIOS", "Grocery",
     {"price": 3.98, "brand": "General Mills", "size": "18oz", "healthy": True},
     {"pairs_well": ["Milk", "Bananas"], "alternatives": ["Lucky Charms", "Corn Flakes"]}),
    
    ("TOY-LEGO-STARWARS", "Toys",
     {"price": 79.99, "brand": "LEGO", "pieces": 500, "age_range": "8-14"},
     {"theme_related": ["Star Wars Action Figures", "SW Lightsaber"], "gift_with": ["Gift Wrap"]}),
]

for product_id, category, attributes, relations in products:
    knowledge = walmart_memory.learn_product_fact(
        product_id=product_id,
        category=category,
        attributes=attributes,
        relations=relations
    )
    print(f"📚 Learned: {product_id} - ${attributes['price']} ({category})")
    print(f"   Relations: {', '.join(relations.keys())}")

In [ ]:
# Test memory retrieval for business decisions
print("\n🔍 Testing Retail Memory Retrieval...\n")

queries = [
    "How did Black Friday go?",
    "Customer complaints in Electronics",
    "Samsung TV related products",
    "Price match procedure",
]

for query in queries:
    print(f"\n❓ Query: '{query}'")
    memories = walmart_memory.retrieve_relevant_memories(query, limit=3)
    
    if memories:
        for i, memory in enumerate(memories, 1):
            if isinstance(memory, RetailEpisode):
                print(f"{i}. [Sale] {memory.content} - Revenue: ${memory.revenue_impact:.2f}")
            elif isinstance(memory, CustomerInteraction):
                print(f"{i}. [Service] {memory.interaction_type}: {memory.resolution}")
            elif isinstance(memory, ProductKnowledge):
                print(f"{i}. [Product] {memory.product_id} - {memory.category}")
            elif isinstance(memory, RetailProcedure):
                print(f"{i}. [Procedure] {memory.procedure_name} - Success: {memory.success_rate:.0%}")
    else:
        print("   No relevant memories found.")

In [ ]:
# Visualize retail intelligence
print("\n📊 Generating Retail Intelligence Dashboard...\n")
walmart_memory.visualize_retail_intelligence()

## 🤖 Part 4: Walmart Assistant with Learning

An AI assistant that learns from store operations and improves over time.

In [ ]:
class WalmartAssistant:
    """AI Assistant for Walmart operations with learning capabilities"""
    
    def __init__(self, name: str, store_id: str):
        self.name = name
        self.store_id = store_id
        self.memory = WalmartMemoryManager(store_id=store_id)
        self.shift_start = datetime.now()
        
    def handle_customer_query(self, query: str, department: str = "General"):
        """Process customer queries using memory"""
        print(f"\n🤖 {self.name}: Processing customer query...")
        print(f"📍 Department: {department}")
        print(f"❓ Query: '{query}'")
        
        # Retrieve relevant memories
        memories = self.memory.retrieve_relevant_memories(query)
        
        response = self._generate_response(query, memories, department)
        
        # Simulate customer satisfaction
        satisfaction = 4.0 if memories else 3.0
        if "price match" in query.lower() and "price_match" in self.memory.procedures:
            satisfaction = 4.5  # Following procedure improves satisfaction
            
        # Remember this interaction
        self.memory.remember_customer_interaction(
            interaction_type="inquiry",
            department=department,
            resolution=response,
            satisfaction=satisfaction
        )
        
        return response
    
    def _generate_response(self, query: str, memories: List, department: str) -> str:
        """Generate response based on memories"""
        query_lower = query.lower()
        
        # Check for price match
        if "price match" in query_lower or "lower price" in query_lower:
            if "price_match" in self.memory.procedures:
                proc = self.memory.procedures["price_match"]
                return (f"I can help with price matching! Here's our process: "
                       f"{', '.join(proc.steps[:3])}. "
                       f"This typically takes {proc.average_time:.0f} minutes.")
        
        # Check for product queries
        for memory in memories:
            if isinstance(memory, ProductKnowledge):
                attrs = memory.attributes
                return (f"I found {memory.product_id}: "
                       f"${attrs.get('price', 'N/A')} - {attrs.get('brand', 'Generic')} "
                       f"in {memory.category}. Located in {department}.")
        
        # Check for Black Friday info
        if "black friday" in query_lower:
            bf_memories = self.memory.seasonal_index.get("Black Friday 2023", [])
            if bf_memories:
                total_sales = sum(m.revenue_impact for m in bf_memories)
                return (f"Black Friday was successful! We processed {len(bf_memories)} "
                       f"major transactions. Our Electronics doorbusters were very popular!")
        
        # Default response
        return f"Let me find someone in {department} to help you with that specific question."
    
    def analyze_shift_performance(self):
        """Analyze performance during shift"""
        shift_duration = (datetime.now() - self.shift_start).total_seconds() / 3600
        
        print(f"\n📊 {self.name}'s Shift Analysis")
        print(f"⏰ Shift duration: {shift_duration:.1f} hours\n")
        
        # Get department insights
        for dept in ["Electronics", "Grocery", "Customer Service"]:
            insights = self.memory.get_department_insights(dept)
            if "total_revenue" in insights:
                print(f"📍 {dept}:")
                print(f"   Revenue: ${insights['total_revenue']:,.2f}")
                print(f"   Transactions: {insights['transaction_count']}")
                if insights['top_products']:
                    top_product = insights['top_products'][0]
                    print(f"   Top item: {top_product[0]}")
                print()
        
        # Customer satisfaction
        print(f"😊 Customer Satisfaction: {self.memory.customer_satisfaction_avg:.2f}/5.0")
        
        # Most used procedures
        if self.memory.procedures:
            print(f"\n🔧 Available Procedures:")
            for name, proc in self.memory.procedures.items():
                print(f"   - {name}: {proc.success_rate:.0%} success rate")
    
    def prepare_for_event(self, event: str):
        """Prepare for upcoming retail event using past memories"""
        print(f"\n🎯 Preparing for {event}...")
        
        # Check if we have memories from similar events
        memories = self.memory.retrieve_relevant_memories(event)
        
        if event.lower() == "black friday" and "black_friday_prep" in self.memory.procedures:
            proc = self.memory.procedures["black_friday_prep"]
            print(f"\n📋 Found preparation procedure with {proc.success_rate:.0%} success rate!")
            print("\nSteps to execute:")
            for i, step in enumerate(proc.steps, 1):
                print(f"  {i}. {step}")
            print(f"\n💰 Expected benefit: ${proc.cost_savings:,.2f}")
            
            # Learn from past events
            past_bf = self.memory.seasonal_index.get("Black Friday 2023", [])
            if past_bf:
                top_categories = defaultdict(float)
                for episode in past_bf:
                    top_categories[episode.department] += episode.revenue_impact
                    
                print("\n📈 Last year's insights:")
                for dept, revenue in sorted(top_categories.items(), 
                                          key=lambda x: x[1], reverse=True)[:3]:
                    print(f"  - {dept}: ${revenue:,.2f} in sales")
        
        return True

# Create Walmart AI assistant
assistant = WalmartAssistant("Sam", "#100")
print(f"✅ Walmart Assistant '{assistant.name}' is ready!")
print(f"🏪 Assigned to Store {assistant.store_id}")

## 🚀 Part 5: Assistant in Action

Let's see our Walmart assistant handle real scenarios.

In [ ]:
# Simulate a busy retail day
print("🌅 Starting retail simulation...\n")

# Morning: Stock popular items
print("=== 8:00 AM - Morning Prep ===")
morning_products = [
    ("GROCERY-MILK-GREATVALUE", "Grocery", 
     {"price": 3.28, "brand": "Great Value", "size": "1 gallon"},
     {"pairs_with": ["Cereal", "Cookies"]}),
    ("PHARM-VITAMINS-C", "Pharmacy",
     {"price": 8.97, "brand": "Equate", "count": 100},
     {"health_category": ["Immune Support", "Daily Vitamins"]}),
]

for prod_id, cat, attrs, rels in morning_products:
    assistant.memory.learn_product_fact(prod_id, cat, attrs, rels)
print("✅ Morning inventory loaded\n")

# Handle customer queries throughout the day
customer_queries = [
    ("9:15 AM", "Where can I find Samsung TVs?", "Electronics"),
    ("10:30 AM", "Do you price match Amazon?", "Customer Service"),
    ("11:45 AM", "Looking for gluten-free cereal", "Grocery"),
    ("2:00 PM", "When is your next Black Friday?", "General"),
    ("3:30 PM", "My Samsung TV from Black Friday isn't working", "Electronics"),
]

for time, query, dept in customer_queries:
    print(f"\n=== {time} ===")
    response = assistant.handle_customer_query(query, dept)
    print(f"💬 Response: {response}")
    time.sleep(0.5)  # Simulate time passing

In [ ]:
# Simulate some sales based on customer interactions
print("\n\n=== Processing Sales Throughout Day ===")

sales = [
    (["GROCERY-MILK-GREATVALUE", "FOOD-CEREAL-CHEERIOS"], 7.26, "regular"),
    (["TV-SAMSUNG-65-Q80A", "Wall Mount", "HDMI Cable"], 949.97, "premium"),
    (["PHARM-VITAMINS-C", "Hand Sanitizer"], 12.47, "health_conscious"),
]

for products, amount, cust_type in sales:
    assistant.memory.remember_sale(products, amount, cust_type)
    print(f"💰 Sale: ${amount:.2f} - {len(products)} items")

# Analyze the shift
print("\n" + "="*50)
assistant.analyze_shift_performance()

In [ ]:
# Prepare for Black Friday using learned knowledge
print("\n" + "="*50)
assistant.prepare_for_event("Black Friday")

## 🎯 Hands-On Exercise: Seasonal Memory System

Create a specialized memory type for seasonal patterns.

In [ ]:
# Exercise: Create a Seasonal Pattern Memory
@dataclass
class SeasonalPattern(RetailMemoryItem):
    """
    TODO: Implement a memory type that learns seasonal patterns
    - Track which products sell well in which seasons
    - Remember temperature/weather correlations
    - Store successful promotional strategies
    """
    season: str = ""               # spring, summer, fall, winter
    year: int = 2024              # Which year
    top_categories: List[str] = field(default_factory=list)
    weather_correlation: Dict[str, float] = field(default_factory=dict)
    promotion_effectiveness: Dict[str, float] = field(default_factory=dict)
    total_revenue: float = 0.0
    
    def __post_init__(self):
        self.memory_type = MemoryType.SEMANTIC  # Patterns are knowledge
        # Seasonal patterns are very important
        self.importance = 0.9
        self.decay_rate = 0.02  # Decay very slowly - yearly relevance

# TODO: Implement a method to analyze seasonal trends
def analyze_seasonal_trends(memory_manager: WalmartMemoryManager, 
                          season: str, year: int) -> SeasonalPattern:
    """Analyze and store seasonal patterns"""
    # Find all episodes from that season
    seasonal_episodes = []
    season_months = {
        "winter": [12, 1, 2],
        "spring": [3, 4, 5],
        "summer": [6, 7, 8],
        "fall": [9, 10, 11]
    }
    
    # Calculate top categories
    category_revenue = defaultdict(float)
    for episode in memory_manager.episodes:
        if episode.timestamp.month in season_months.get(season, []):
            category_revenue[episode.department] += episode.revenue_impact
    
    top_cats = sorted(category_revenue.items(), key=lambda x: x[1], reverse=True)
    
    # Create seasonal pattern
    pattern = SeasonalPattern(
        content=f"Seasonal pattern for {season} {year}",
        timestamp=datetime.now(),
        importance=0.9,
        season=season,
        year=year,
        top_categories=[cat for cat, _ in top_cats[:5]],
        total_revenue=sum(category_revenue.values()),
        weather_correlation={"temperature": 0.7, "precipitation": -0.3},
        promotion_effectiveness={"BOGO": 0.85, "clearance": 0.92}
    )
    
    return pattern

# Test your implementation
print("🍂 Analyzing Fall 2023 patterns...")
fall_pattern = analyze_seasonal_trends(assistant.memory, "fall", 2023)
print(f"\nTop categories: {', '.join(fall_pattern.top_categories[:3])}")
print(f"Total revenue impact: ${fall_pattern.total_revenue:,.2f}")
print(f"Best promotion: Clearance at {fall_pattern.promotion_effectiveness['clearance']:.0%} effectiveness")

## 🔗 Part 6: Intelligent Retail Decision Making

Using all memory types to make smart retail decisions.

In [ ]:
class RetailDecisionEngine:
    """Makes data-driven retail decisions using memory"""
    
    def __init__(self, memory_manager: WalmartMemoryManager):
        self.memory = memory_manager
        
    def recommend_inventory_action(self, product_id: str) -> Dict[str, Any]:
        """Recommend inventory decisions based on memory"""
        print(f"\n🔮 Analyzing inventory for {product_id}...")
        
        recommendation = {
            "product": product_id,
            "action": "hold",
            "confidence": 0.5,
            "reasoning": [],
            "projected_impact": 0.0
        }
        
        # Check product knowledge
        if product_id in self.memory.product_knowledge:
            product = self.memory.product_knowledge[product_id]
            
            # Check sales history
            sales_count = sum(1 for e in self.memory.episodes 
                            if product_id in e.products_involved)
            
            if sales_count > 5:
                recommendation["action"] = "increase_stock"
                recommendation["confidence"] = 0.8
                recommendation["reasoning"].append(f"High sales volume: {sales_count} recent sales")
                
            # Check if frequently bought with other items
            if "frequently_bought_with" in product.relations:
                related = product.relations["frequently_bought_with"]
                if len(related) > 2:
                    recommendation["confidence"] += 0.1
                    recommendation["reasoning"].append(f"Cross-sells with {len(related)} products")
                    
            # Check seasonality
            current_month = datetime.now().month
            if current_month in [11, 12]:  # Holiday season
                if product.category in ["Electronics", "Toys"]:
                    recommendation["action"] = "increase_stock"
                    recommendation["confidence"] = min(0.95, recommendation["confidence"] + 0.2)
                    recommendation["reasoning"].append("Holiday season - high demand category")
                    
        else:
            recommendation["action"] = "research_product"
            recommendation["reasoning"].append("No product history - need more data")
            
        # Calculate projected impact
        if recommendation["action"] == "increase_stock":
            avg_sale = np.mean([e.revenue_impact for e in self.memory.episodes 
                              if product_id in e.products_involved] or [50])
            recommendation["projected_impact"] = avg_sale * 10  # Assume 10 more sales
            
        return recommendation
    
    def optimize_store_operations(self, time_of_day: int) -> Dict[str, Any]:
        """Optimize operations based on patterns"""
        print(f"\n⚙️ Optimizing operations for {time_of_day}:00...")
        
        optimization = {
            "time": f"{time_of_day}:00",
            "staff_allocation": {},
            "focus_areas": [],
            "procedures_to_review": []
        }
        
        # Analyze activity patterns
        hour_activity = defaultdict(int)
        dept_activity = defaultdict(int)
        
        for episode in self.memory.episodes:
            hour = episode.timestamp.hour
            if abs(hour - time_of_day) <= 1:  # Within 1 hour window
                dept_activity[episode.department] += 1
                
        # Recommend staff allocation
        total_activity = sum(dept_activity.values())
        if total_activity > 0:
            for dept, count in dept_activity.items():
                percentage = (count / total_activity) * 100
                optimization["staff_allocation"][dept] = f"{percentage:.0f}%"
                
        # Identify focus areas
        if time_of_day in [8, 9, 10]:  # Morning
            optimization["focus_areas"] = ["Restocking", "Price updates", "Online pickup prep"]
        elif time_of_day in [11, 12, 13, 14]:  # Midday
            optimization["focus_areas"] = ["Customer service", "Checkout efficiency", "Lunch coverage"]
        elif time_of_day in [17, 18, 19]:  # Evening rush
            optimization["focus_areas"] = ["All registers open", "Quick restock of essentials", "Customer flow management"]
            
        # Check procedures that might be needed
        if self.memory.customer_satisfaction_avg < 4.0:
            optimization["procedures_to_review"].append("customer_service_recovery")
            
        return optimization

# Create decision engine
decision_engine = RetailDecisionEngine(assistant.memory)

# Test inventory recommendations
print("📦 Testing Inventory Recommendations...")
products_to_analyze = ["TV-SAMSUNG-65-Q80A", "TOY-LEGO-STARWARS", "UNKNOWN-PRODUCT"]

for product in products_to_analyze:
    rec = decision_engine.recommend_inventory_action(product)
    print(f"\n📊 {product}:")
    print(f"   Action: {rec['action']}")
    print(f"   Confidence: {rec['confidence']:.0%}")
    print(f"   Reasoning: {'; '.join(rec['reasoning'])}")
    if rec['projected_impact'] > 0:
        print(f"   Projected impact: +${rec['projected_impact']:,.2f}")

# Test operational optimization
print("\n\n🕐 Testing Operational Optimization...")
for hour in [9, 13, 18]:
    opt = decision_engine.optimize_store_operations(hour)
    print(f"\nOptimization for {opt['time']}:")
    print(f"  Staff allocation: {opt['staff_allocation']}")
    print(f"  Focus areas: {', '.join(opt['focus_areas'])}")

## 📊 Summary and Key Takeaways

### What You've Learned:

1. **Retail-Specific Memory Architecture**:
   - Working Memory: Current store state and active customers
   - Episodic Memory: Sales transactions and customer interactions
   - Semantic Memory: Product knowledge and relationships
   - Procedural Memory: Operational workflows with ROI tracking

2. **Walmart Business Intelligence**:
   - Revenue impact tracking per memory
   - Customer satisfaction monitoring
   - Department-level insights
   - Seasonal pattern recognition

3. **Learning from Retail Operations**:
   - Product relationship discovery (frequently bought together)
   - Procedure effectiveness measurement
   - Customer service improvement tracking
   - Black Friday and seasonal preparation

4. **Data-Driven Decision Making**:
   - Inventory recommendations based on sales patterns
   - Staff allocation optimization by hour
   - Procedure selection based on success rates
   - Cross-selling opportunities

### 🚀 Real-World Applications:

This memory system could power:
- **Smart Inventory Management**: Predictive restocking
- **Customer Service Bots**: Context-aware assistance
- **Revenue Optimization**: Data-driven pricing and promotions
- **Operational Excellence**: Learning from successful procedures

### 💡 Challenge Exercise:

Implement a **Competitive Intelligence Memory** that:
1. Tracks competitor pricing and promotions
2. Learns which price matches drive sales
3. Identifies market trends and opportunities
4. Suggests competitive strategies

This would give Walmart AI agents true market awareness!

### 📈 Business Impact:

With proper implementation, this system could:
- Increase customer satisfaction by 15%
- Reduce inventory waste by 20%
- Improve checkout efficiency by 25%
- Boost seasonal revenue by 10%

Ready for Module 3? We'll connect these agents to real databases and APIs! 🚀